In [26]:
import sqlite3 as sq3
import pandas as pd

In [27]:
#creat DB + load data
data = pd.read_csv("udemy_courses.csv")
db = sq3.connect("udemy_courses_db.db")
data.to_sql("udemy_courses", db, index = False)
cur = db.cursor()

In [31]:
#Q1 - TOP 5 free courses by subject
#is_paid column => True = 1, False = 0

cur.execute('SELECT * FROM\
            (SELECT "course_title", "course_id", "is_paid", "subject", "num_subscribers", ROW_NUMBER()\
            OVER (PARTITION BY "subject" ORDER BY "num_subscribers" DESC) AS rank\
            FROM "udemy_courses"\
            WHERE "is_paid" = 0)\
            WHERE rank < 6')
    
TOP5 = pd.DataFrame(cur.fetchall())
TOP5.columns = ["course_title", "course_id", "is_paid", "subject", "num_subscribers", "rank"]
TOP5 = TOP5.set_index("rank")
display(TOP5)

,course_title,course_id,is_paid,subject,num_subscribers
rank,,,,,
1,Bitcoin or How I Learned to Stop Worrying and ...,49798,0,Business Finance,65576
2,Accounting in 60 Minutes - A Brief Introduction,48841,0,Business Finance,56659
3,Stock Market Investing for Beginners,133536,0,Business Finance,50855
4,Introduction to Financial Modeling,151668,0,Business Finance,29167
5,Forex Basics,97338,0,Business Finance,22344
1,Photoshop In-Depth: Master all of Photoshop's ...,348116,0,Graphic Design,53851
2,Figure Drawing From Life Using The Reilly Tech...,17349,0,Graphic Design,47811
3,Professional Logo Design in Adobe Illustrator,399938,0,Graphic Design,44044
4,Learn to Design a Letterhead - A Beginners Course,22445,0,Graphic Design,24687


In [32]:
# Q2 - TOP10 most popular courses
cur.execute('SELECT "course_id", "course_title"\
            FROM "udemy_courses"\
            ORDER BY "num_subscribers" DESC\
            LIMIT 10')
    
TOP10 = pd.DataFrame(cur.fetchall())
TOP10.columns = ["course_id", "course_title"]
TOP10["rank"] = range(1, 11)
TOP10 = TOP10.set_index("rank")
display(TOP10)

,course_id,course_title
rank,,
1,41295,Learn HTML5 Programming From Scratch
2,59014,Coding for Entrepreneurs Basic
3,625204,The Web Developer Bootcamp
4,173548,Build Your First Website in 1 Week with HTML5 ...
5,764164,The Complete Web Developer Course 2.0
6,19421,Free Beginner Electric Guitar Lessons
7,473160,Web Design for Web Developers: Build Beautiful...
8,94430,Learn Javascript & JQuery From Scratch
9,130064,Practical PHP: Master the Basics and Code Dyna...


In [35]:
# Q3 - best year by number of courses

cur.execute('SELECT strftime("%Y","published_timestamp") AS "year", COUNT(*) AS "count"\
            FROM "udemy_courses"\
            GROUP BY "year"\
            ORDER BY "count" DESC')
    
best_year = pd.DataFrame(cur.fetchall())
best_year.columns = ["year", "count"]
display(best_year)

,year,count
0,2016,1206
1,2015,1014
2,2017,715
3,2014,491
4,2013,202
5,2012,45
6,2011,5
